In [1]:
import pandas as pd

In [2]:
#Global paramenters
#API URLs from RxNorm
base_url = "https://rxnav.nlm.nih.gov/REST/rxcui/"
base_url_2 = "https://rxnav.nlm.nih.gov/REST/"
base_url_3 = "https://rxnav.nlm.nih.gov/REST/rxcui.json?name="
#RxNorm methods
getAllRelatedInfo = "/allrelated.json"
getRelatedByType = "/related.json?tty=IN"
#dataframe
df = pd.DataFrame()

In [3]:
#Method to get drug with only the opioid ingredient
def query_self(name_of_drug):
    rd_df = pd.read_json(base_url+name_of_drug+getRelatedByType)
    rd_df = pd.DataFrame(rd_df.relatedGroup.conceptGroup)
    rd_df = pd.DataFrame(rd_df.conceptProperties[0])
    return rd_df

#Method to get all drugs with an opioid ingredient
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    rd_df = pd.read_json(base_url+name_of_drug+getAllRelatedInfo)
    rd_df = pd.DataFrame(rd_df.allRelatedGroup.conceptGroup)
    rd_df = rd_df.loc[rd_df["tty"] != "DF"].reset_index(drop=True)
    rd_df = rd_df.loc[rd_df["tty"] != "DFG"].reset_index(drop=True)
    while row < rd_df.shape[0]:
        if(pd.isna(rd_df.conceptProperties[row]) is True):
            row+=1
        else:
            new_df = pd.DataFrame(rd_df.conceptProperties[row])
            if df.empty:
                df = new_df
            else:
                df = pd.concat([df, new_df]).reset_index(drop=True)
            row += 1
    return df

In [4]:
#Read in ingredients from excel file
drug_ingredients = pd.read_excel("Antidepressants_Ingredients.xlsx", header=None)
drugs = []
#Get rxcui for each ingredient
for drug in drug_ingredients[0]:
    ingredient = pd.read_json(base_url_3+drug)
    drugs.append(ingredient.idGroup.rxnormId[0])

#loop array to get complete list of drugs
df = pd.DataFrame()
for drug in drugs:
    self_df = query_self(drug)
    ingredient = self_df["name"].values[0]
    self_df["ingredient"] = ingredient
    merg_df = query_drug(drug)
    merg_df["ingredient"] = ingredient
    df = pd.concat([df, self_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
    df = pd.concat([df, merg_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df

,rxcui,name,synonym,tty,language,suppress,umlscui,ingredient
0,94,5-hydroxytryptophan,oxitriptan,IN,ENG,N,,5-hydroxytryptophan
1,1293415,Somnicin,,BN,ENG,N,,5-hydroxytryptophan
2,1008781,5-hydroxytryptophan / vitamin B6,,MIN,ENG,N,,5-hydroxytryptophan
3,1293412,5-hydroxytryptophan / magnesium oxide / melato...,,MIN,ENG,N,,5-hydroxytryptophan
4,1945211,5-hydroxytryptophan / melatonin,,MIN,ENG,N,,5-hydroxytryptophan
...,...,...,...,...,...,...,...,...
1559,1439811,vortioxetine 5 MG,,SCDC,ENG,N,,vortioxetine
1560,1439839,vortioxetine 15 MG,,SCDC,ENG,N,,vortioxetine
1561,1439807,vortioxetine Oral Tablet,,SCDF,ENG,N,,vortioxetine
1562,1439805,vortioxetine Oral Product,,SCDG,ENG,N,,vortioxetine


In [5]:
#Reduce to usefull columns
df = df[['rxcui', 'name',"tty", "ingredient"]]
df

,rxcui,name,tty,ingredient
0,94,5-hydroxytryptophan,IN,5-hydroxytryptophan
1,1293415,Somnicin,BN,5-hydroxytryptophan
2,1008781,5-hydroxytryptophan / vitamin B6,MIN,5-hydroxytryptophan
3,1293412,5-hydroxytryptophan / magnesium oxide / melato...,MIN,5-hydroxytryptophan
4,1945211,5-hydroxytryptophan / melatonin,MIN,5-hydroxytryptophan
...,...,...,...,...
1559,1439811,vortioxetine 5 MG,SCDC,vortioxetine
1560,1439839,vortioxetine 15 MG,SCDC,vortioxetine
1561,1439807,vortioxetine Oral Tablet,SCDF,vortioxetine
1562,1439805,vortioxetine Oral Product,SCDG,vortioxetine


In [6]:
#Export
df.to_excel("New_Complete_Antidepressants_List.xlsx", index=False)

In [10]:
df_list = pd.read_excel("Complete_Antidepressants_List.xlsx")
df_list = df_list[['Code', 'Description','TTY']]
df_list.rename(columns= {'Code' : 'rxcui', 'Description': 'name'}, inplace = True)
df_list

,rxcui,name,TTY
0,1000048,doxepin 10 MG Oral Capsule,SCD
1,1000054,doxepin 10 MG/ML Oral Solution,SCD
2,1000058,doxepin 100 MG Oral Capsule,SCD
3,1000064,doxepin 150 MG Oral Capsule,SCD
4,1000070,doxepin 25 MG Oral Capsule,SCD
...,...,...,...
202,846386,lithium orotate 5 MG Oral Capsule,SCD
203,966787,doxepin 3 MG Oral Tablet,SCD
204,966793,doxepin 6 MG Oral Tablet,SCD
205,1439840,vortioxetine 15 MG Oral Tablet,SCD


In [11]:
df_list["ingredients"] = None
for i, row in df_list.iterrows():
    try:
        df_active_ing = query_self(str(df_list.loc[i, "rxcui"]))
        df_list.at[i, 'ingredients'] = list(df_active_ing['name'])
    except AttributeError:
        continue


In [12]:
df_list['ingredients'] = df_list['ingredients'].fillna('Obsolete Error')
df_list

,rxcui,name,TTY,ingredients
0,1000048,doxepin 10 MG Oral Capsule,SCD,[doxepin]
1,1000054,doxepin 10 MG/ML Oral Solution,SCD,[doxepin]
2,1000058,doxepin 100 MG Oral Capsule,SCD,[doxepin]
3,1000064,doxepin 150 MG Oral Capsule,SCD,[doxepin]
4,1000070,doxepin 25 MG Oral Capsule,SCD,[doxepin]
...,...,...,...,...
202,846386,lithium orotate 5 MG Oral Capsule,SCD,[lithium]
203,966787,doxepin 3 MG Oral Tablet,SCD,[doxepin]
204,966793,doxepin 6 MG Oral Tablet,SCD,[doxepin]
205,1439840,vortioxetine 15 MG Oral Tablet,SCD,[vortioxetine]


In [13]:
df.to_excel("Complete_Antidepressants_List_with_Ingredients.xlsx", index=False)

In [14]:
#Identify new drugs found
same_drug_count = 0
new_drug_count = 0
drug_not_found_count = 0
same_drug = pd.DataFrame()
new_drug = pd.DataFrame()
for code in df['rxcui']:
    drug_name = df['name'].where(df['rxcui'] == code).dropna().values[0]
    if df_list.isin([int(code)]).any().any() == True:
        same_drug_count += 1
        same_drug = pd.concat([same_drug, df.loc[df['rxcui']==code]]).reset_index(drop=True)
    else:
        new_drug_count += 1
        new_drug = pd.concat([new_drug, df.loc[df['rxcui']==code]]).reset_index(drop=True)
print('Same drug count: ', same_drug_count)
print('New drug count: ', new_drug_count)

Same drug count:  188
New drug count:  1376


In [11]:
#export
same_drug.to_excel("Same_Antidepressant_Medications.xlsx", index=False)
new_drug.to_excel("New_Antidepressant_Added.xlsx", index=False)

In [15]:
absent_drugs = pd.DataFrame()
count = 0
#for every code from the combind excel list
for code in df_list['rxcui']:
    if df.isin([str(code)]).any().any() == True:
        count+=1
    else:
        absent_drugs = pd.concat([absent_drugs, df_list.loc[df_list['rxcui']==code]]).reset_index(drop=True)
print('Count: ', count)
absent_drugs

Count:  188


,rxcui,name,TTY,ingredients
0,859186,selegiline hydrochloride 5 MG Oral Capsule,SCD,[selegiline]
1,859190,selegiline hydrochloride 1.25 MG Disintegratin...,SCD,[selegiline]
2,859193,selegiline hydrochloride 5 MG Oral Tablet,SCD,[selegiline]
3,865206,24 HR selegiline 0.25 MG/HR Transdermal System,SCD,[selegiline]
4,865210,24 HR selegiline 0.375 MG/HR Transdermal System,SCD,[selegiline]
5,865214,24 HR selegiline 0.5 MG/HR Transdermal System,SCD,[selegiline]
6,197889,lithium carbonate 300 MG Oral Capsule,SCD,[lithium carbonate]
7,197890,lithium carbonate 300 MG Oral Tablet,SCD,[lithium carbonate]
8,197891,lithium carbonate 300 MG Extended Release Oral...,SCD,[lithium carbonate]
9,197892,lithium carbonate 450 MG Extended Release Oral...,SCD,[lithium carbonate]


In [18]:
#find the status of the not found drugs
df_status = pd.DataFrame()
index = 0
for code in absent_drugs['rxcui']:
    getRxcuiHistoryStatus = 'rxcui/{}/historystatus.json'.format(code)
    df_type = pd.read_json(base_url_2+getRxcuiHistoryStatus)
    df_type = pd.DataFrame(df_type.rxcuiStatusHistory.metaData, index=[0])
    df_type['rxcui'] = code
    df_status = pd.concat([df_status, df_type]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_status

,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate,rxcui
0,Active,RXNORM,092009,,YES,092009,,,859186
1,Active,RXNORM,092009,,YES,092009,,,859190
2,Active,RXNORM,092009,,YES,092009,,,859193
3,Active,RXNORM,102009,,YES,102009,,,865206
4,Active,RXNORM,102009,,YES,102009,,,865210
5,Active,RXNORM,102009,,YES,102009,,,865214
6,Active,RXNORM,042005,,YES,042005,,,197889
7,Active,RXNORM,042005,,YES,042005,,,197890
8,Active,RXNORM,042005,,YES,042005,,,197891
9,Active,RXNORM,042005,,YES,042005,,,197892


In [19]:
df_not_found = absent_drugs.merge(df_status, on='rxcui')
df_not_found = df_not_found[['rxcui', 'name','TTY', 'status',"ingredients"]]
df_not_found

,rxcui,name,TTY,status,ingredients
0,859186,selegiline hydrochloride 5 MG Oral Capsule,SCD,Active,[selegiline]
1,859190,selegiline hydrochloride 1.25 MG Disintegratin...,SCD,Active,[selegiline]
2,859193,selegiline hydrochloride 5 MG Oral Tablet,SCD,Active,[selegiline]
3,865206,24 HR selegiline 0.25 MG/HR Transdermal System,SCD,Active,[selegiline]
4,865210,24 HR selegiline 0.375 MG/HR Transdermal System,SCD,Active,[selegiline]
5,865214,24 HR selegiline 0.5 MG/HR Transdermal System,SCD,Active,[selegiline]
6,197889,lithium carbonate 300 MG Oral Capsule,SCD,Active,[lithium carbonate]
7,197890,lithium carbonate 300 MG Oral Tablet,SCD,Active,[lithium carbonate]
8,197891,lithium carbonate 300 MG Extended Release Oral...,SCD,Active,[lithium carbonate]
9,197892,lithium carbonate 450 MG Extended Release Oral...,SCD,Active,[lithium carbonate]


In [20]:
df_active=df_not_found.loc[df_not_found['status']== 'Active'].reset_index(drop=True)
df_obsolete = df_not_found.loc[df_not_found['status']== 'Obsolete'].reset_index(drop=True)

In [21]:
#export
df_not_found.to_excel("Status_of_Not_found_Antidepressant.xlsx", index=False)
df_active.to_excel("Active_Not_Found_Antidepressant.xlsx", index=False)
df_obsolete.to_excel("Obsolete_Not_Found_Antidepressant.xlsx", index=False)